<a href="https://colab.research.google.com/github/misuTAMC/Legal-Text-Retrieval-/blob/main/legal_text_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [11]:
!kaggle datasets download -d hariwh0/zaloai2021-legal-text-retrieval


Dataset URL: https://www.kaggle.com/datasets/hariwh0/zaloai2021-legal-text-retrieval
License(s): unknown
 66% 70.0M/106M [00:00<00:00, 226MB/s]
100% 106M/106M [00:00<00:00, 278MB/s] 


In [12]:
!unzip zaloai2021-legal-text-retrieval.zip


Archive:  zaloai2021-legal-text-retrieval.zip
  inflating: legal_corpus.json       
  inflating: legal_corpus_hashmap.csv  
  inflating: legal_corpus_legend.csv  
  inflating: legal_corpus_merged_u256.csv  
  inflating: legal_corpus_merged_u369.csv  
  inflating: legal_corpus_original.csv  
  inflating: legal_corpus_splitted.csv  
  inflating: public_test_question.json  
  inflating: public_test_sample_submission.json  
  inflating: stopwords.txt           
  inflating: train_qna.csv           
  inflating: train_question_answer.json  


In [13]:
import json

with open("legal_corpus.json", "r", encoding='utf-8') as f:
    law = json.load(f)

print(law[0:2])
# {'id': '1', 'text': 'Điều 1. Phạm vi điều chỉnh ...'}


[{'law_id': '01/2009/tt-bnn', 'articles': [{'article_id': '1', 'title': 'Điều 1. Phạm vi áp dụng', 'text': 'Thông tư này hướng dẫn tuần tra, canh gác bảo vệ đê Điều trong mùa lũ đối với các tuyến đê sông được phân loại, phân cấp theo quy định tại Điều 4 của Luật Đê Điều.'}, {'article_id': '2', 'title': 'Điều 2. Tổ chức lực lượng', 'text': '1. Hàng năm trước mùa mưa, lũ, Ủy ban nhân dân cấp xã nơi có đê phải tổ chức lực lượng lao động tại địa phương để tuần tra, canh gác đê và thường trực trên các điếm canh đê hoặc nhà dân khu vực gần đê (đối với những khu vực chưa có điếm canh đê), khi có báo động lũ từ cấp I trở lên đối với tuyến sông có đê (sau đây gọi tắt là lực lượng tuần tra, canh gác đê).\n2. Lực lượng tuần tra, canh gác đê được tổ chức thành các đội, do Ủy ban nhân dân cấp xã ra quyết định thành lập; từ 01 đến 02 kilômét đê thành lập 01 đội; mỗi đội có từ 12 đến 18 người, trong đó có 01 đội trưởng và 01 hoặc 02 đội phó. Danh sách thành viên đội tuần tra, canh gác đê được niêm yế

#**XÂY DỰNG DANH SÁCH CÂU TRUY VẤN VÀ DOC**

In [14]:
corpus = []

for sample in law:
  for article in sample['articles']:
    corpus.append({
        'title': article['title'],
        'text': article['text']
    })



In [15]:
text_corpus=[]
for sample in corpus:
  text_corpus.append(sample['text'])

#**XÂY DỰNG TEXT NORMALIZATION**

In [16]:
f = open("stopwords.txt")
stopwords=f.read()
stopwords_list=stopwords.replace('\n',',').split(',')

In [17]:
# import re

# def clean_text(text):
#     if isinstance(text, str):
#         text = text.replace('\xa0', ' ')
#         text = re.sub(r'\s+', ' ', text)
#         return text.strip()
#     elif isinstance(text, list):
#         return [clean_text(item) for item in text]
#     elif isinstance(text, dict):
#         return {k: clean_text(v) for k, v in text.items()}
#     else:
#         return text

# cleaned_data = clean_text(corpus)


In [18]:
import re
def clean_text(text):
  if isinstance(text,str):
    text=text.replace('\xa0',' ')
    text=re.sub(r'\s+',' ',text)
    return text.strip()

In [19]:
a=[]
for text in text_corpus:
  text=clean_text(text)
  a.append(text)

In [20]:
text_corpus=a

In [21]:
import re
def remove_stopwords(text, stopwords):
    # cụm dài xóa trước
    for sw in sorted(stopwords, key=lambda x: -len(x)):
        # 1 cái boundary \blà\b
        pattern = r'\b' + re.escape(sw) + r'\b'
        #thay đúng từ là <=> \blà\b bằng '' trong text
        text = re.sub(pattern, '', text)
    return re.sub(r'\s+', ' ', text).strip()


In [22]:
text = "Tôi là một kỹ sư của công ty và làm việc chăm chỉ."
stopwords = ["là", "của", "và", "một"]

cleaned = remove_stopwords(text, stopwords)
print(cleaned)


Tôi kỹ sư công ty làm việc chăm chỉ.


In [23]:
def tokenize(text):
  return text.split()

In [24]:
# import string
# remove_special_char=string.punctuation

# def text_normalization(text):
#     # 1. Chuyển về chữ thường
#     doc = text.lower()
#     # 2. Xóa dấu câu (kể cả những dấu không có trong string.punctuation)
#     doc = re.sub(r'[{}]+'.format(re.escape(remove_special_char)), ' ', doc)
#     doc = re.sub(r'[“”‘’–—…]', ' ', doc)  # xóa dấu ngoặc kép kiểu đặc biệt trong văn bản luật
#     # 3. Xóa số (nếu bạn muốn)
#     doc = re.sub(r'\d+', ' ', doc)
#     # 4. Xóa nhiều khoảng trắng liên tiếp thành 1 khoảng trắng
#     doc = re.sub(r'\s+', ' ', doc).strip()
#     # 5. Xóa stopwords
#     doc = remove_stopwords(doc, stopwords_list)
#     return doc

# # Áp dụng cho corpus list
# def normalize_corpus(corpus):
#     return [text_normalization(doc) for doc in corpus]

In [27]:
import re
import string

remove_special_char = string.punctuation

def text_normalization(text):
    # 1. Chuyển về chữ thường
    doc = text.lower()
    # 2. Xóa dấu câu
    doc = re.sub(r'[{}]+'.format(re.escape(remove_special_char)), ' ', doc)
    doc = re.sub(r'[“”‘’–—…]', ' ', doc)
    # 3. Xóa số
    doc = re.sub(r'\d+', ' ', doc)
    # 4. Xóa khoảng trắng thừa
    doc = re.sub(r'\s+', ' ', doc).strip()
    # 5. Xóa stopwords
    doc = remove_stopwords(doc, stopwords_list)
    return doc
def normalize_corpus(corpus):
    return [text_normalization(doc) for doc in corpus]

In [28]:
text_corpus=normalize_corpus(text_corpus[:1000])


#**XÂY DỰNG BỘ DICTIONARY**

In [29]:
def create_dictionary(corpus):
    dic = set()
    for doc in corpus:
        # normalized_doc = text_normalization(doc)
        tokenized_doc = tokenize(doc)
        dic.update(tokenized_doc)  # Thêm tất cả token vào set (tự loại bỏ trùng lặp)
    return list(dic)  # Trả ra list nếu cần


In [30]:
dictionary=create_dictionary(text_corpus)
dictionary

['dôi',
 'nứa',
 'qh',
 'nạn',
 'hòa',
 'khám',
 'tl',
 'standard',
 'lốp',
 'xưởng',
 'phông',
 'tia',
 'vạch',
 'games',
 'ngoại',
 'lắp',
 'tt',
 'vamc',
 'vi',
 'túc',
 'hạ',
 'kiến',
 'lụt',
 'viện',
 'lassa',
 'sa',
 'dẻo',
 'dạy',
 'tạp',
 'đường',
 'soi',
 'olympic',
 'sơ',
 'xét',
 'cầm',
 'mặc',
 'xi',
 'chênh',
 'củng',
 'xóa',
 'trú',
 'ls',
 'đòi',
 'chiết',
 'nguyên',
 'nhập',
 'bầu',
 'mua',
 'interest',
 'bắc',
 'bến',
 'đổi',
 'asiad',
 'cai',
 'd',
 'ccvc',
 'bất',
 'ngư',
 'xứ',
 'ẩm',
 'truất',
 'lam',
 'sóng',
 'phong',
 'lựa',
 'xe',
 'thếp',
 'thiệu',
 'vp',
 'kệ',
 'khắc',
 'option',
 'qđpdpa',
 'lạc',
 'lỗ',
 'đích',
 'gián',
 'hối',
 'an',
 'para',
 'lộn',
 'hiểm',
 'vọng',
 'tuỳ',
 'pháp',
 'hải',
 'mời',
 'lặp',
 'độc',
 'đẳng',
 'phụ',
 'thước',
 'ngón',
 'gật',
 'còi',
 'thay',
 'cháu',
 'dạng',
 'bô',
 'bảng',
 'gv',
 'quan',
 'bình',
 'y',
 'kỷ',
 'quầy',
 'khử',
 'đảm',
 'tụng',
 'lưu',
 'say',
 'tccb',
 'túi',
 'giao',
 'sáp',
 'phim',
 'gửi',
 'diễn',

In [31]:
text_corpus[17]

'thông tư chỉnh quan đại diện cộng hòa xã hội chủ nghĩa việt nam bao quan đại diện ngoại giao quan đại diện lãnh quan đại diện tổ chức quốc tế quy định luật quan đại diện cộng hòa xã hội chủ nghĩa việt nam'

#**XÂY DỰNG HÀM VECTOR HOÁ VĂN BẢN**

In [32]:
def vectorize(text,dictionary):
  word_count_dict={word:0 for word in dictionary}
  normalized_text=text_normalization(text)
  tokens=tokenize(normalized_text)
  for token in tokens:
    try:
      word_count_dict[token]+=1
    except:
      pass
  vector=list(word_count_dict.values())
  return vector

In [33]:
vec=vectorize(text_corpus[1],dictionary)

In [34]:
for idx,val in enumerate(vec):
  if val==3:
    print(idx)

534
769
934
969
990
1331


In [35]:
idx=[69,164,371,1231,1271,1374]
print([dictionary[i] for i in idx])

['kệ', 'cư', 'khảo', 'giường', 'súng', 'tựu']


#**XÂY DỰNG DOC-TERM**

In [36]:
for index,doc in enumerate(text_corpus):
  print('index : ',index,'\n')
  print(doc)


index :  0 

thông tư hướng tuần tra canh gác bảo vệ đê mùa lũ tuyến đê sông phân phân quy định luật đê
index :  1 

hàng mùa mưa lũ ủy ban nhân dân xã đê tổ chức lực lao động địa phương tuần tra canh gác đê trực điếm canh đê dân khu vực đê khu vực điếm canh đê báo động lũ i trở tuyến sông đê gọi tắt lực tuần tra canh gác đê lực tuần tra canh gác đê tổ chức thành đội ủy ban nhân dân xã quyết định thành lập kilômét đê thành lập đội đội đội trưởng đội phó danh sách thành viên đội tuần tra canh gác đê niêm yết điếm canh đê địa bàn phân công lũ bão diễn biến phức tạp kéo uỷ ban nhân dân xã quyết định bổ sung thành viên đội tuần tra canh gác đê
index :  2 

khoẻ tháo vát khả năng đảm đương công nhọc mưa to gió đêm tối tinh thần trách nhiệm đựng gian khổ khắc phục quen sông bơi kiến thức kinh nghiệm hộ đê phòng chống lụt bão
index :  3 

chấp hành phân công ban huy phòng chống lụt bão hướng chuyên môn nghiệp vụ quan chuyên trách quản lý đê tuần tra canh gác trực điếm canh đê báo động lũ i tr

In [37]:
def create_doc_term_matrix(corpus, dictionary):
    dtm = {}
    for index, doc in enumerate(corpus):
        normalized_doc = text_normalization(doc)
        vector_doc = vectorize(normalized_doc, dictionary)
        dtm[(index, doc)] = vector_doc
    return dtm  # nhớ nằm ngoài vòng for


In [38]:
corpus[0]['title']

'Điều 1. Phạm vi áp dụng'

In [39]:
doc1='reading math books'
doc2='linear algebra book'
doc3='learn how ai learns'
doc4='reading books'
dic=[doc1,doc2,doc3,doc4]
dic

['reading math books',
 'linear algebra book',
 'learn how ai learns',
 'reading books']

In [40]:
new_dic=[]
for doc in dic:
  normal=text_normalization(doc)
  tokens=tokenize(normal)
  for token in tokens:
    if token not in new_dic:
      new_dic.append(token)
print(new_dic)

['reading', 'math', 'books', 'linear', 'algebra', 'book', 'learn', 'how', 'learns']


In [41]:
import numpy as np
doc1v=vectorize(doc1,new_dic)
doc2v=vectorize(doc2,new_dic)
doc3v=vectorize(doc3,new_dic)
doc4v=vectorize(doc4,new_dic)
print(np.array([new_dic,doc1v,doc2v,doc3v,doc4v]))

[['reading' 'math' 'books' 'linear' 'algebra' 'book' 'learn' 'how'
  'learns']
 ['1' '1' '1' '0' '0' '0' '0' '0' '0']
 ['0' '0' '0' '1' '1' '1' '0' '0' '0']
 ['0' '0' '0' '0' '0' '0' '1' '1' '1']
 ['1' '0' '1' '0' '0' '0' '0' '0' '0']]


In [42]:
dtm=create_doc_term_matrix(dic,new_dic)
dtm

{(0, 'reading math books'): [1, 1, 1, 0, 0, 0, 0, 0, 0],
 (1, 'linear algebra book'): [0, 0, 0, 1, 1, 1, 0, 0, 0],
 (2, 'learn how ai learns'): [0, 0, 0, 0, 0, 0, 1, 1, 1],
 (3, 'reading books'): [1, 0, 1, 0, 0, 0, 0, 0, 0]}

In [43]:
doc_term_mat=create_doc_term_matrix(text_corpus,dictionary)

#**COSINE SIMILARITY**

In [44]:
from scipy import spatial
def cosine_similarity (vec1, vec2) :
  return 1 - spatial.distance.cosine(vec1, vec2)

#**XÂY DỰNG HÀM QUERY**

In [45]:
def ranking_relevant_legal(query,dictionary,doc_term_matrix):
  normalized_query=text_normalization(query)
  vector_query=vectorize(normalized_query,dictionary)
  score_of_all=[]
  for doc_in4,doc_vec in doc_term_matrix.items():
    cosine_similar=cosine_similarity(vector_query,doc_vec)
    score_of_all.append((cosine_similar,doc_in4))
  score_of_all.sort(reverse=True)

  return score_of_all

In [46]:
text_normalization('Hàng năm trước mùa mưa')

'hàng mùa mưa'

In [52]:
query_lst = [' đ) Cho phép người khác sử dụng tài liệu ký gửi tại Lưu trữ lịch sử, nhưng không được xâm hại an ninh quốc gia, lợi ích của Nhà nước']
top_k = 10

for query in query_lst:
    scores = ranking_relevant_legal(query, dictionary, doc_term_mat)
    print(f"Query : {query}")
    if not scores:
      print("Không tìm thấy tài liệu liên quan.")
    else:
      for idx in range(top_k):
          doc_score = scores[idx][0]
          doc_content = scores[idx][1][1]
          print(f"Top {idx + 1}; Score : {doc_score:.4f}")
          print(doc_content)
          print('\n')


Query :  đ) Cho phép người khác sử dụng tài liệu ký gửi tại Lưu trữ lịch sử, nhưng không được xâm hại an ninh quốc gia, lợi ích của Nhà nước
Top 1; Score : 0.6779
tài liệu gia đình dòng gọi phục vụ hoạt động thực tiễn nghiên cứu khoa học lịch sử quốc gia xã hội đăng ký phông lưu trữ quốc gia việt nam a gia phả tộc phả sắc phong tài liệu tiểu sử b thảo viết in bút tích công trình nghiên cứu khoa học tác thư trao đổi c phim ảnh băng đĩa ghi âm ghi hình tài liệu điện tử d công trình viết đ ấn phẩm tài liệu sưu tầm lưu trữ lịch sử đăng ký trách nhiệm xác định tài liệu phông lưu trữ quốc gia việt nam quy định khoản tài liệu quyền a đăng ký tài liệu lưu trữ lịch sử hướng giúp đỡ kỹ thuật bảo quản phát huy tài liệu quy định khoản b quyết định hiến tặng ký gửi tài liệu lưu trữ lịch sử c thỏa thuận mua tài liệu d ưu tiên tài liệu hiến tặng đ phép tài liệu ký gửi lưu trữ lịch sử xâm hại an ninh quốc gia lợi ích quyền lợi ích hợp pháp quan tổ chức e khen thưởng quy định pháp luật tài liệu nghĩa v